此檔案進行包含 Dropout 的 ResNet50 訓練（針對全部資料集，無 Validation）。主要流程包括：

1. **模型架構**：採用 ResNet-50 作為基底模型，並在最後的全連接層加入 Dropout（p=0.5）以防止 Overfitting。部分層（如 `layer1` 和 `layer2`）在訓練過程中被凍結，以加快訓練速度並保留已學習的特徵。

2. **數據處理與增強**：
   - 使用 `datasets.py` 中的 `MultiLabelDataset` 類別來處理標籤為多個類別的圖像數據。  
   - 應用多種數據增強技術，如隨機裁剪、旋轉、色彩抖動等，來提高模型的泛化能力。  
   - 使用 Mixup 技術，將不同樣本進行加權混合。

3. **訓練配置**：
   - 使用 `BCEWithLogitsLoss` 作為損失函數來處理多標籤問題，並配置 Adam Optimizer。  

4. **評估指標**：訓練過程中計算 F1-score、Precision、Recall、MAP 等指標。

5. **模型儲存與可視化**：
   - 每個訓練週期（Epoch）後保存模型參數，並記錄訓練損失與指標。  
   - 使用 TensorBoard 來可視化訓練過程中的損失和指標變化，方便分析模型學習情況。


In [ ]:
import pandas as pd
import numpy as np
import json
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score, average_precision_score
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader
from torchvision import transforms, models
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torch.cuda.amp import GradScaler, autocast
from torch.utils.tensorboard import SummaryWriter
from PIL import Image
import os

from datasets import MultiLabelDataset

class MultiLabelResNet50(nn.Module):
    def __init__(self, num_classes, pretrained=True):
        super(MultiLabelResNet50, self).__init__()
        self.model = models.resnet50(pretrained=pretrained)

        for name, param in self.model.named_parameters():
            if "layer1" in name or "layer2" in name:
                param.requires_grad = False

        in_features = self.model.fc.in_features
        self.model.fc = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(in_features, num_classes)
        )

    def forward(self, x):
        return self.model(x)

def calculate_metrics(y_true, y_pred, y_scores):
    metrics = {}
    metrics['f1'] = f1_score(y_true, y_pred, average='micro', zero_division=0)
    metrics['precision'] = precision_score(y_true, y_pred, average='micro', zero_division=0)
    metrics['recall'] = recall_score(y_true, y_pred, average='micro', zero_division=0)
    try:
        metrics['roc_auc'] = roc_auc_score(y_true, y_pred, average='micro')
    except:
        metrics['roc_auc'] = 0.0
    try:
        ap_per_class = average_precision_score(y_true, y_scores, average=None)
        metrics['map'] = np.mean(ap_per_class)
    except Exception as e:
        print("Error calculating MAP:", e)
        metrics['map'] = 0.0
    return metrics

def mixup_data(x, y, alpha=1.0):
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1
    batch_size = x.size()[0]
    if torch.cuda.is_available():
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

def main():
    with open('class_mapping.json', 'r') as f:
        class_mapping = json.load(f)

    ground_truth = pd.read_csv('ground_truth_probabilities.csv')
    if 'filename' not in ground_truth.columns:
        raise ValueError("CSV 文件中缺少 'filename' 列")

    print(f"Number of training samples: {len(ground_truth)}")

    train_transforms = transforms.Compose([
        transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(20),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    img_dir = 'train_data'

    train_dataset = MultiLabelDataset(
        dataframe=ground_truth,
        img_dir=img_dir,
        class_mapping=class_mapping,
        transform=train_transforms
    )

    train_loader = DataLoader(
        train_dataset,
        batch_size=64,
        shuffle=True,
        num_workers=4,
        pin_memory=True
    )

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device:", device)

    num_classes = len(class_mapping)
    model = MultiLabelResNet50(num_classes=num_classes, pretrained=True).to(device)

    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50)

    scaler = GradScaler()
    writer = SummaryWriter('runs/multi_label_experiment')

    num_epochs = 80
    train_losses = []
    train_f1_scores = []
    train_map_scores = []

    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        print("-" * 30)

        model.train()
        running_loss = 0.0
        all_preds = []
        all_labels = []

        for inputs, labels in tqdm(train_loader, desc="Training"):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            inputs, labels_a, labels_b, lam = mixup_data(inputs, labels, alpha=1.0)

            with autocast():
                outputs = model(inputs)
                loss = mixup_criterion(criterion, outputs, labels_a, labels_b, lam)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            running_loss += loss.item() * inputs.size(0)

            preds = torch.sigmoid(outputs).detach().cpu().numpy()
            all_preds.append(preds)
            all_labels.append(labels.detach().cpu().numpy())

        epoch_loss = running_loss / len(train_loader.dataset)
        all_preds = np.vstack(all_preds)
        all_labels = np.vstack(all_labels)
        preds_binary = (all_preds > 0.5).astype(int)
        train_metrics = calculate_metrics(all_labels, preds_binary, all_preds)

        train_losses.append(epoch_loss)
        train_f1_scores.append(train_metrics['f1'])
        train_map_scores.append(train_metrics['map'])

        print(f"Training Loss: {epoch_loss:.4f} | F1: {train_metrics['f1']:.4f} | MAP: {train_metrics['map']:.4f}")

        model_path = f'model_epoch_{epoch+1}.pth'
        torch.save(model.state_dict(), model_path)
        print(f"Model for epoch {epoch+1} saved as {model_path}.")

        scheduler.step()

    torch.save(model.state_dict(), 'final_model_resnet50_dropout.pth')
    print("Final model saved as 'final_model_resnet50_dropout.pth'.")

    writer.close()

if __name__ == '__main__':
    main()


Number of training samples: 37866
Using device: cuda


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNe

Epoch 1/80
------------------------------


Training:   0%|          | 0/592 [00:00<?, ?it/s]<ipython-input-10-c089759c8cce>:153: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Training: 100%|██████████| 592/592 [08:32<00:00,  1.16it/s]


Training Loss: 0.0946 | F1: 0.0918 | MAP: 0.0429
Model for epoch 1 saved as model_epoch_1.pth.
Epoch 2/80
------------------------------


...

Training:   0%|          | 0/592 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
<ipython-input-10-c089759c8cce>:153: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Training: 100%|██████████| 592/592 [08:51<00:00,  1.11it/s]


Training Loss: 0.0457 | F1: 0.4285 | MAP: 0.2904
Model for epoch 77 saved as model_epoch_77.pth.
Epoch 78/80
------------------------------


Training:   0%|          | 0/592 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
<ipython-input-10-c089759c8cce>:153: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Training: 100%|██████████| 592/592 [08:43<00:00,  1.13it/s]


Training Loss: 0.0451 | F1: 0.4225 | MAP: 0.2798
Model for epoch 78 saved as model_epoch_78.pth.
Epoch 79/80
------------------------------


Training:   0%|          | 0/592 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
<ipython-input-10-c089759c8cce>:153: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Training:   8%|▊         | 48/592 [00:42<04:47,  1.89it/s]